In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

In [2]:
path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/all/nc/*.nc'
flist = glob.glob(path)
# print(len(flist))

In [3]:
ds=xr.open_mfdataset(flist, concat_dim='time')

ds.dims
ds.coords
print(ds.Band1.shape)

(51, 16781, 17568)


/home/meganmason/Documents/projects/thesis/analysis/virtual_envs/data_analysis/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/home/meganmason/Documents/projects/thesis/analysis/virtual_envs/data_analysis/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  coords=coords)


In [4]:
date=[]

for f in sorted(flist):
#     print(f)
    dt_str = f.split("/")[-1] #splits on / and saves the last one
    dt_str = "".join([c for c in dt_str if c.isnumeric()]) #grabs numeric values
    dt = pd.to_datetime(dt_str[:8]) #convert to datetime, grabs digits for dates only
#     print('file:',dt.date())
    date.append(dt.date())

time_var = pd.to_datetime(date)
ds['DATE'] = (['time'], time_var)
ds = ds.assign_coords(time=ds.DATE)
# ds.sel(time='2015')
ds = ds.sel(time='2018')
ds

<xarray.Dataset>
Dimensions:              (time: 2, x: 17568, y: 16781)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
  * time                 (time) datetime64[ns] 2018-04-23 2018-05-28
Data variables:
    transverse_mercator  (time) |S1 b'' b''
    Band1                (time, y, x) float64 dask.array<shape=(2, 16781, 17568), chunksize=(1, 16781, 17568)>
    DATE                 (time) datetime64[ns] 2018-04-23 2018-05-28
Attributes:
    GDAL_AREA_OR_POINT:     Area
    GDAL_TIFFTAG_SOFTWARE:  MATLAB 8.5, Mapping Toolbox 4.1
    Conventions:            CF-1.5
    GDAL:                   GDAL 2.2.3, released 2017/11/20
    history:                Thu Aug 01 17:17:04 2019: GDAL CreateCopy( /home/...

In [5]:
t=time.time()

# variance in snow depth
varsn = ds.Band1.var(dim='time') #compute variance on every pixel in the time dimention 
varsn = np.where(varsn>.001, varsn, np.nan) # filters everything greater than 1mm (i.e gives alpha bg)
print(varsn)
# fig = plt.figure(figsize=(15, 10))
# plt.imshow(varsn, cmap='jet', origin='lower', vmin=.05, vmax=1.0) #[::-1,:] - reverses it
# plt.title('snow depth [3m] variance, 2013-2018')
# plt.colorbar()
# plt.savefig('figs/var_13-16', dpi=600)

tt=time.time()
print('runtime =', (tt-t)/60)

/home/meganmason/Documents/projects/thesis/analysis/virtual_envs/data_analysis/lib/python3.6/site-packages/dask/array/reductions.py:387: RuntimeWarning: invalid value encountered in true_divide
  u = total / n
/home/meganmason/Documents/projects/thesis/analysis/virtual_envs/data_analysis/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


MemoryError: 